# Text Mining Group Project
## EN-FI Corpus

##### TOC for Implemented Metrics.

---

In [1]:
# Necessary Installs.
#!pip install rouge

In [2]:
# Imports
import pandas as pd
import numpy as np
from collections import Counter
from rouge import Rouge
import string
from nltk.translate import chrf_score
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import StandardScaler

Load Dataset

In [3]:
df1 = pd.read_csv("scores.csv")

In [4]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6748 entries, 0 to 6747
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   source       6748 non-null   object 
 1   reference    6748 non-null   object 
 2   translation  6748 non-null   object 
 3   z-score      6748 non-null   float64
 4   avg-score    6748 non-null   float64
 5   annotators   6748 non-null   int64  
dtypes: float64(2), int64(1), object(3)
memory usage: 316.4+ KB


In [5]:
df1.head()

,source,reference,translation,z-score,avg-score,annotators
0,"You can turn yourself into a pineapple, a dog ...","Voit muuttaa itsesi ananasta, koirasta tai Roy...","Voit muuttaa itsesi ananakseksi, koiraksi tai ...",-0.286195,34.20,5
1,Also shot were three men: two 29-year-olds and...,Myös ammuttiin kolme miestä: kaksi 29-vuotiait...,Myös kolmea miestä ammuttiin: kahta 29-vuotias...,0.547076,58.40,5
2,The information is stored at the cash register...,Tiedot tallennetaan kassakoneisiin joka tapauk...,Tiedot kuitenkin tallentuvat kassoilla joka ta...,1.122476,74.60,5
3,Xinhua says that there were traces of hydrochl...,"Xinhua kertoo, että Xinyin näytteestä oli sunn...","Xinhua kertoo, että Xinyin sunnuntaina antamas...",0.383095,53.60,5
4,"MacDonald, who was brought on board CBC's comm...",Voitaisiin kuulla CBD: n kommenttitiimin toimi...,"MacDonaldin, joka tuli CBC:n selostajatiimiin ...",-0.493065,32.25,4


---
### PreProcessing

In [6]:
# Check for empty or sparse reference / translation, and drop them.
for column in ["source","reference","translation"]:
    print(column)
    bad_idx = [idx for idx in np.where(df1[column].str.len()<=2)[0]]
    if bad_idx != None:
        print(df1.iloc[bad_idx])
    print(f"Bad idx: {bad_idx}")
    df1 = df1.drop(index=bad_idx)

source
Bad idx: []
reference
Bad idx: []
translation
Bad idx: []


In [7]:
# Create two dataframes. One is unprocessed, the other is preprocessed to remove punctuation and be lowercased.
# (Future note: maybe also remove stopwords?)
df_u = df1.copy()
df_p = df1.copy()

for x in ["source","reference","translation"]:
    # lowercase.
    df_p[x] = df1[x].str.lower()
    # Remove punct.
    df_p[x] = df1[x].map(lambda s: s.translate(str.maketrans("","",string.punctuation))
                           .lower()
                          )
    
df_dict = {"df_u":df_u,"df_p":df_p}

In [8]:
## Initialize a scaler for later.
scaler = StandardScaler()

---
### Metrics

--- 
Bleu Metric

In [9]:
def my_first_BLEU(reference,translation):
    """
    Expects lists of strings for both reference and translation.
    Returns the score 
    """
    
    # Let word be every unique word in the translation.
    # Can be done by setting up a Counter object.
    t_c = Counter(word_tokenize(translation))
    words = sorted(t_c)
    
    refs_c = Counter(word_tokenize(reference))
    
    # Let Covered be the minimum amt of times a word appears in the reference, compared to R(w).
    # Let D(word) be how many times a unique word appears in the candidate translation.
    # Let R(word) be the largest numer of times the word appears in any one reference.
    
    covered = 0
    
    for word in words:
        covered += min(t_c[word],refs_c[word])

    

    # Let total be the number of words in translation.
    total = sum(t_c.values())
    
    BLEU_score = covered / total
    
    return BLEU_score


In [10]:
bleu_scores_list = ["basic bleu"]

for df in list(df_dict.values()):
    for key in bleu_scores_list:
        # Apply the function to get a column of the scores.
        df[(key+" score")] = df.apply(lambda row: my_first_BLEU(row["reference"],row["translation"]),axis=1)
        # Also add a z score column.
        df[(key+ "_zscore")] = scaler.fit_transform(df[(key+" score")].to_numpy().reshape(-1,1)).flatten()

---
Rouge metric as described in

https://towardsdatascience.com/the-ultimate-performance-metric-in-nlp-111df6c64460

And
https://pypi.org/project/rouge-metric/

In [11]:
rouge = Rouge()

In [12]:
# Example Cell
model_out = df1["translation"][0]
reference = df1["reference"][0]
# The get scores method returns three metrics, F1 score, p precision and recall r.
# For each unigram,bigram and Longest sequence.
rouge.get_scores(model_out,reference)

[{'rouge-1': {'f': 0.5217391254442345, 'p': 0.5, 'r': 0.5454545454545454},
  'rouge-2': {'f': 0.2857142807256236, 'p': 0.2727272727272727, 'r': 0.3},
  'rouge-l': {'f': 0.5217391254442345, 'p': 0.5, 'r': 0.5454545454545454}}]

In [13]:
# Get scores.
# For every df considered;
for df in list(df_dict.values()):
    # For the entire model, model_out and reference need to be lists of strings.
    model_out = df["translation"].to_list()
    reference = df["reference"].to_list()
    rouge_scores = rouge.get_scores(model_out,reference)
    # For each of the scores calculated, output a new column in the df with the f1 scores.
    for key in rouge_scores[0].keys():
        df[(key+" score")] = pd.Series([score[key]["f"] for score in rouge_scores])
        # Also add a z score column.
        df[(key+ "_zscore")] = scaler.fit_transform(df[(key+" score")].to_numpy().reshape(-1,1)).flatten()

---
#### chrF metric

Check the paper here: https://www.aclweb.org/anthology/W15-3049.pdf

The general formula for the CHRF score is:

`CHRFBeta = (1 + Beta**2) * ((chrP * chrR) / (Beta**2*chrP + chrR))`

where:
* chrP is the percentage of n-grams in the hypothesis which have a counterpart in the reference.
* chrR is the percentage of character n-grams in the reference which are also present in the hypothesis.
* Beta is a parameter which assigns beta times more importance to recall than to precision (if beta == 1, they have the same importance).

In [14]:
# I was surprised, but this works exactly like it's intended. Makes a new column with the chrF score for each row of the df.
# The default n-gram values are min == 1, max == 6. 
# The default beta is 3.

# Moreover, it is worthwhile to mention chrf uses its own tokenization with whitespaces.
# All parameters to test chrf scores with. feel free to play around with this and test out different combinations.
# Note: this takes a few minutes to run.
min_len = [1,2]
max_len = [6,10]
beta = [1,3]

for df in list(df_dict.values()):
    chrf_scores=[]
    for min_l in min_len:
        for max_l in max_len:
            for b in beta:
                append_str = "chrf_b" + str(b) + "_n" + str(min_l) + str(max_l)
                chrf_scores.append(append_str)
                df[append_str] = df.apply(lambda row: chrf_score.sentence_chrf(row["reference"],row["translation"],min_len=min_l,max_len=max_l,beta=b),axis=1)
                # Also add a z score column.
                df[(append_str+ "_zscore")] = scaler.fit_transform(df[append_str].to_numpy().reshape(-1,1)).flatten()

df_p.loc[:,chrf_scores]

,chrf_b1_n16,chrf_b3_n16,chrf_b1_n110,chrf_b3_n110,chrf_b1_n26,chrf_b3_n26,chrf_b1_n210,chrf_b3_n210
0,0.637506,0.647060,0.561316,0.569916,0.594947,0.603933,0.529207,0.537385
1,0.551969,0.564371,0.369415,0.377803,0.488911,0.500031,0.314099,0.321328
2,0.484037,0.499739,0.369613,0.381859,0.409852,0.423308,0.315686,0.326300
3,0.668674,0.798229,0.569547,0.682215,0.641981,0.767278,0.543704,0.652130
4,0.372883,0.378628,0.267856,0.272022,0.281105,0.285473,0.205198,0.208424
...,...,...,...,...,...,...,...,...
6743,0.399885,0.437072,0.273764,0.299357,0.311441,0.340579,0.210615,0.230448
6744,0.415380,0.372602,0.279145,0.250280,0.337263,0.302350,0.220609,0.197660
6745,0.344243,0.359218,0.219900,0.229526,0.246990,0.257917,0.152055,0.158837
6746,0.435735,0.387187,0.304516,0.270404,0.358353,0.318217,0.246947,0.219112


---
### Comparison of Applied Metrics
Because the numeric system used for all of these can be different, the best way to compare them is by checking the correlation with the annotator's scores.

In [15]:
# Initialize a dict to be transformed to a df later, for score comparison.
corr_list = ["pearson","kendall"]
scores_dict = {}

for df_name in df_dict.keys():
    for corr in corr_list:
        scores_dict[corr+"_"+df_name] = []
    
scores_index = []

In [16]:
# Thankfully, Pandas has a corr method. Use it on standardized scores obtained previously.

# for each declared corr method, compute the corr between each computed metric and the avg-score column for each considered df.
for df_name in df_dict.keys():
    for corr in corr_list:
        for key in rouge_scores[0].keys():
            scores_dict[corr+"_"+df_name].append(df_dict[df_name].loc[:,(key+ "_zscore")].corr(df_dict[df_name].loc[:,"z-score"],method=corr))
        for chrf_score in set(chrf_scores):
            scores_dict[corr+"_"+df_name].append(df_dict[df_name].loc[:,(chrf_score+"_zscore")].corr(df_dict[df_name].loc[:,"z-score"],method=corr))
        for bleu_score in bleu_scores_list:
            scores_dict[corr+"_"+df_name].append(df_dict[df_name].loc[:,(key+ "_zscore")].corr(df_dict[df_name].loc[:,"z-score"],method=corr))


# Build also a list that will be used to create the index for the scores dataframe.
scores_index.extend(list(rouge_scores[0].keys()))
scores_index.extend(chrf_scores)
scores_index.extend(bleu_scores_list)

In [17]:
scores_df = pd.DataFrame(scores_dict,index=scores_index)
scores_df

,pearson_df_u,kendall_df_u,pearson_df_p,kendall_df_p
rouge-1,0.518201,0.341341,0.532356,0.350394
rouge-2,0.420468,0.292348,0.433422,0.300453
rouge-l,0.509038,0.334333,0.521265,0.341528
chrf_b1_n16,0.578030,0.390627,0.572972,0.386378
chrf_b3_n16,0.583569,0.392867,0.578090,0.388538
chrf_b1_n110,0.607690,0.403670,0.602130,0.399336
chrf_b3_n110,0.557968,0.373855,0.552100,0.369059
chrf_b1_n26,0.604377,0.401822,0.599336,0.397675
chrf_b3_n26,0.558419,0.371978,0.551958,0.366980
chrf_b1_n210,0.578574,0.379403,0.572085,0.374211


In [18]:
for column in scores_df.columns:
    print(f"Max in {column}:")
    print("{}, with {}".format(scores_df[column].argmax(),scores_df[column].max()))


Max in pearson_df_u:
5, with 0.6076896353710804
Max in kendall_df_u:
5, with 0.4036696282333412
Max in pearson_df_p:
5, with 0.6021298061849121
Max in kendall_df_p:
5, with 0.3993358124975348
